In [1]:
import warnings
warnings.filterwarnings("ignore")
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)


## Commodities Expiration Functions






In [2]:
import os,sys
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as pdr
import datetime
import pytz
import pathlib
from dateutil.relativedelta import *
import pandas_market_calendars as pmc
import pathlib
from dashapp import dashapp2 as dashapp
import os,sys
import requests
import io
import subprocess
import re
import xml.etree.ElementTree as ET
import xmltodict
import zipfile
import chrome_driver_upload as cdu
from pandas.tseries.holiday import USFederalHolidayCalendar
import sel_scrape as sc
import db_info
import pdb

from tqdm import tqdm,tqdm_notebook

In [3]:
# important constants used below
MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}
TIMEZONE = 'US/Eastern'
CHROME_DOWNLOADED=False
THIS_DECADE = int(str(int(datetime.datetime.now().year))[2])*10
DO_DATABASE_COMPARE=False

In [4]:
# DOWNLOAD_CHROMEDRIVER = False
# if DOWNLOAD_CHROMEDRIVER and not CHROME_DOWNLOADED:
#     cdu.download_chromedriver()
#     CHROME_DOWNLOADED=True
# sac = sc.SelScrape(headless=False,driver_name='chrome')#,profile_path=pp)
# sac.goto('https://www.cmegroup.com/tools-information/quikstrike/options-calendar.html')


In [5]:
# uk_holidays = open('expiration_data/uk_holidays.csv').readlines()
# uksplit = [','.join(l.split(',')) for l in  uk_holidays]
# fio = io.StringIO()
# fio.writelines(uksplit)
# fio.seek(0)
# df_ukh = pd.read_csv(fio)
# def ukh_to_yyyymmdd(month_day,year):
#     md = month_day.strip()
#     d = datetime.datetime.strptime(f'{md} {year}', '%B %d %Y')
#     return d
# #     return d.strftime("%Y-%m-%d")

# year_cols = [c for c in df_ukh.columns.values if '20' in str(c)]
# for c in year_cols:
#     df_ukh[c] = df_ukh[c].apply(lambda s:ukh_to_yyyymmdd(s,c))
# uk_holidays = sorted(df_ukh[year_cols].values.reshape(-1))


In [6]:
# df_ukh

In [7]:
# bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
# bday_uk = pd.offsets.CustomBusinessDay(holidays=uk_holidays)

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    next_monday,next_monday_or_tuesday,USMartinLutherKingJr, USPresidentsDay, \
    GoodFriday, USMemorialDay, USLaborDay, USThanksgivingDay,EasterMonday

class USTradingCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
        USLaborDay,
        USThanksgivingDay,
        Holiday('Christmas', month=12, day=25, observance=nearest_workday),
    ]

bday_us = pd.offsets.CustomBusinessDay(calendar=USTradingCalendar())

class UKTradingCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
        GoodFriday,
        EasterMonday,
        Holiday('EarlyMayHoliday',month=5,day=1,offset=pd.DateOffset(weekday=MO(1))),
        Holiday('SpringBankHoliday',month=6,day=1,offset=pd.DateOffset(weekday=MO(-1))),
        Holiday('SummerBankHoliday',month=9,day=1,offset=pd.DateOffset(weekday=MO(-1))),
        Holiday('StAndrewsDay',month=11,day=30),
        Holiday('Christmas', month=12, day=25, observance=next_monday),
        Holiday('Christmas', month=12, day=26, observance=next_monday_or_tuesday),
    ]    
    
bday_uk = pd.offsets.CustomBusinessDay(calendar=UKTradingCalendar())

2020-07-01 16:13:38,955 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [127]:
prng = pd.period_range('1/1/2011', '1/1/2012', freq='M')
ps = pd.Series(np.random.randn(len(prng)), prng)
ps

2011-01    2.186849
2011-02    1.386757
2011-03   -0.466509
2011-04    0.541812
2011-05   -2.526971
2011-06   -0.513648
2011-07    0.134549
2011-08   -1.198023
2011-09   -1.657903
2011-10    0.634970
2011-11   -0.006940
2011-12   -0.715316
2012-01    1.650521
Freq: M, dtype: float64

In [126]:
pi = pd.period_range('2016-01-01', periods=3, freq='M')
[p[1] for p in [pi.astype('datetime64')[0:2],pi.astype('datetime64[D]')[0:2]]]

[Timestamp('2016-02-01 00:00:00', freq='MS'),
 Timestamp('2016-02-01 00:00:00', freq='MS')]

In [ ]:
# given that
# when you use a quarterly Period as the start arg of the period_range method, 
# and a freq arg of "M", why does the period_range method produce the first Period
# object as a Period which starts on the last month of the

In [8]:
def get_thanksgiving_dt(year):    
    iyyyy = int(year)
    holidays = USTradingCalendar().holidays(start=f'{iyyyy}-11-01', end=f'{iyyyy}-11-30', return_name=True)
    df_holidays = holidays.to_frame()
    df_holidays =df_holidays.reset_index()
    df_holidays.columns=['hdate','hname']
    return df_holidays[['giving' in v.lower() for v  in df_holidays.hname.values]].hdate.values[0]




In [9]:
_DAY_INDEX = ['MON','TUE','WED','THU','FRI','SAT','SUN']
def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    f = f'W-{_DAY_INDEX[target_weekday]}'
    dr = pd.date_range(datetime.datetime(year,month,1), periods=nth_occurrence, freq=f)[-1]
    return dr
#     # get dayofweeks of year,month,1
#     weekday_01 = datetime.datetime(year,month,1).weekday()
#     if weekday_01 <= target_weekday:
#         day_of_month_of_first_occurence = target_weekday - weekday_01
#         day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
#     else:
#         day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
#     return datetime.datetime(year,month,day_of_month_of_nth_occurence)




def get_ES_options_expiry(symbol):
    '''
    3rd friday of month of symbol
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return get_nth_weekday(year,month,4,3)


def get_E6_options_expiry(symbol):
    monthcode_yy = symbol[2:]
    next_month = DICT_MONTH_CODE[monthcode_yy[0]] + 1
    year = 2000 + int(monthcode_yy[1:])
    if next_month>12:
        next_month = 1
        year += 1
    return datetime.datetime(year,next_month,1) - 7*bday_us

def get_LO_options_expiry(symbol):
    return get_CL_options_expiry('CL'+symbol[-3:])

def get_CL_options_expiry(symbol):
    '''
    Trading terminates 7 business days before the 26th calendar of the month prior to the contract month.
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,26) - 7*bday_us

def get_ON_options_expiry(symbol):
    return get_NG_options_expiry('NG'+symbol[-3:])

def get_LNE_options_expiry(symbol):
    return get_NG_options_expiry('NG'+symbol[-3:])

def get_NG_options_expiry(symbol):
    monthcode_yy = symbol[-3:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return datetime.datetime(year,month,1) - 4*bday_us

def get_GE_options_expiry(symbol):
    """
    Quarterly:
    The second London bank business day before the third Wednesday 
    of the contract month. Trading in expiring contracts ceases at 
    11:00 a.m. London Time on the last trading day
    
    Serialy:
    The Friday immediately preceding the third Wednesday of the contract month.
    """
    monthcode_yy = symbol[-3:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    wed_3 = get_nth_weekday(year,month,2,3)
    mc = monthcode_yy[0]
    dr = pd.date_range(datetime.datetime(year,month,1), periods=3, freq='W-WED')
    if mc in ['H','M','U','Z']:        
        r = [d - 2*bday_uk for d in dr][-1]
    else:
        r = [d - datetime.timedelta(5) for d in dr][-1] + 1*bday_uk - 1*bday_uk
    return r

def get_CB_options_expiry(symbol):
    '''
    This is the spec for the CME Brent, but it matches ICE.
    Trading terminates the 4th last London business day of 
    the month, 2 months prior to the contract month 
    except for the February contract month which 
    terminates the 5th last London business day of the 
    month, 2 months prior to the contract month.  
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month - 1
    if month<1:
        month = 12 + month
        year = year - 1
    days_to_subtract = 4
    if monthcode_yy[0] =='G':
        days_to_subtract = 5
    elif monthcode_yy[0] == 'F':
        days_to_subtract = 3
#     elif monthcode_yy == 'N22':
#         days_to_subtract = 7
    return datetime.datetime(year,month,1,0,0) - days_to_subtract * bday_uk


def get_GC_options_expiry(symbol):
    """
    Trading terminates at 12:30 p.m. CT on the 4th last business day of the month
    prior to the contract month. 
    If the 4th last business day occurs on a Friday 
    or the day before a holiday, 
    trading terminates on the prior business day.
    """
    exp_dt = get_ON_options_expiry(symbol)
    r = exp_dt
    if exp_dt.weekday()==4:
        r =  exp_dt - 1*bday_us
    td1 = datetime.timedelta(1)
    is_day_before_thanksgiv = (get_thanksgiving_dt(r.year) - (r + td1)).days == 0 
    if is_day_before_thanksgiv:
        r = r - td1
    return r

def get_ZS_options_expiry(symbol):
    """
    Trading terminates on Friday which precedes, by at least 2 business days, 
    the last business day of the month prior to the contract month.
    """
    monthcode_yy = symbol[-3:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    r =  datetime.datetime(year,month,1) - 3*bday_us
#     pdb.set_trace()
    dow = r.weekday()
    if dow>=4:
        days_to_subtract = dow-4
    else:
        # find the first friday before r
        days_to_subtract = dow-4+7
    r = r - datetime.timedelta(days_to_subtract)
    return r+1*bday_us - 1*bday_us
        

DICT_PRODUCT = {
    'E6':get_E6_options_expiry,
    'ES':get_ES_options_expiry,
    'CL':get_CL_options_expiry,
    'NG':get_NG_options_expiry,
    'LO':get_LO_options_expiry,
    'ON':get_ON_options_expiry,
    'LNE':get_LNE_options_expiry,
    'LN':get_ON_options_expiry,
    'CB':get_CB_options_expiry,
    'RB':get_ON_options_expiry,
    'HO':get_ON_options_expiry,
    'OB':get_ON_options_expiry,
    'OH':get_ON_options_expiry,
    'GC':get_GC_options_expiry,
    'OG':get_GC_options_expiry,
    'SI':get_GC_options_expiry,
    'SO':get_GC_options_expiry,
    'ZS':get_ZS_options_expiry,
    'OZS':get_ZS_options_expiry,
    'ZC':get_ZS_options_expiry,
    'OZC':get_ZS_options_expiry,
    'ZW':get_ZS_options_expiry,
    'OZW':get_ZS_options_expiry,
    'ZL':get_ZS_options_expiry,
    'OZL':get_ZS_options_expiry,
    'GE':get_GE_options_expiry,
}

    
def get_expiry(symbol):
    product = symbol[:-3]
    f = DICT_PRODUCT[product]
    expiry =  f(symbol)
    if (expiry.weekday()==0) and (expiry.month==12 and expiry.day==27):
        expiry=expiry + 1*bday_us
    return expiry


def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))

def yyyymmdd_from_dt(dt):
    y = int(dt.year)
    m = int(dt.month)
    d = int(dt.day)
    return y*100*100 + m*100 + d

def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365


In [11]:
get_expiry('GEZ20')#,df_cme_expiry[df_cme_expiry.option_symbol=='GEZ20'].option_expiry.values

Timestamp('2020-12-14 00:00:00')

In [12]:
#pd.DateOffset(weekday=MO(2))
from pandas.tseries.holiday import TU,MO
pd.Timestamp(2020,7,1) + pd.DateOffset(weekday=TU(-1))


Timestamp('2020-06-30 00:00:00')

### Methods to extract all current CME expirations from CME website

In [13]:
def get_full_underlying_symbol(r,decade=2,underlying_col='Underlying Symbol'):
    underyear = int(re.findall('[0-9]{1,2}$',r[underlying_col])[0])
    if underyear > 9:
        return r[underlying_col]
    under_symbol_no_year = r[underlying_col][:-1]
    return under_symbol_no_year + str(decade*10+underyear)

def get_full_option_symbol(r,decade=2,option_col='Option Symbol',
                           underlying_col='Underlying Symbol'):
    optyear = int(re.findall('[0-9]{1,2}$',r[option_col])[0])
    if optyear>9:
        return r[option_col]

    # we have a one character option year
    # first see if this option month is different than the underlying month
    op_monthcode = re.findall('[A-Z][0-9]{1,2}$',r[option_col])[0][0]
    un_monthcode = re.findall('[A-Z][0-9]{1,2}$',r[underlying_col])[0][0]
    underyear = int(re.findall('[0-9]{1,2}$',r[underlying_col])[0])
    if (underyear > 9) and (op_monthcode==un_monthcode):
        return r[option_col][:-1] + str(underyear)
    opt_symbol_no_year = r[option_col][:-1]
    return opt_symbol_no_year + str(decade*10+optyear)



In [14]:
def get_cme_expiries():    
    df_eee_all = None
    for i in tqdm_notebook(range(1,7)):
        url = f'https://cmegroup-tools.quikstrike.net/User/Export/CME/ExpirationCalendar.aspx?GroupId={i}'
        df_eee_temp = pd.read_csv(url)
        if df_eee_all is None:
            df_eee_all = df_eee_temp.copy()
        else:
            df_eee_all = df_eee_all.append(df_eee_temp,ignore_index=True)
        df_eee_all.index = list(range(len(df_eee_all)))

    oexps = [s[:10] for s in df_eee_all['Option Expiration Date (CT)'].values]
    exp_dts = [datetime.datetime.strptime(str_date,'%m/%d/%Y') for str_date in oexps]
    df_eee_all['option_expiry'] = exp_dts

    unexps = ['01/01/1900' if 'nan' in str(s).lower() else s[:10] for s in df_eee_all['Underlying Expiration Date (CT)'].values]
    unexp_dts = [datetime.datetime.strptime(str_date,'%m/%d/%Y') for str_date in unexps]
    df_eee_all['underlying_expiry'] = unexp_dts

    df_eee_all['underlying_symbol'] = df_eee_all.apply(get_full_underlying_symbol,axis=1)
    opsym_lambda = lambda r:get_full_option_symbol(r,underlying_col='underlying_symbol')
    df_eee_all['option_symbol'] = df_eee_all.apply(opsym_lambda,axis=1)
    return df_eee_all


In [15]:
df_cme_expiry = get_cme_expiries()

In [16]:
df_cme_expiry.query("option_symbol in ['OZSZ20','LOF30']")

,Option First Trade Date,Option Expiration Date (CT),Option Product,Option Symbol,Underlying Symbol,Underlying Expiration Date (CT),option_expiry,underlying_expiry,underlying_symbol,option_symbol
204,08/24/2020,11/20/2020 13:20:00,Soybean Options,OZSZ0,ZSF1,01/14/2021 12:01:00,2020-11-20,2021-01-14,ZSF21,OZSZ20
604,06/10/2019,12/14/2029 12:30:00,Crude Oil Options,LOF0,CLF30,12/19/2029 13:30:00,2029-12-14,2029-12-19,CLF30,LOF30


### Test get_expiry with know expirations in the future

In [17]:
for op_sym in tqdm_notebook(['LO','LNE','ES','OB','OH','OG','SO','OZS','OZC','OZW','OZL','GE']):
    df_lo = df_cme_expiry[df_cme_expiry.option_symbol.str.contains(f'^{op_sym}[FGHJKMNQUVXZ][2-4][0-9]')][['option_symbol','underlying_symbol','option_expiry','underlying_expiry']]
    if len(df_lo)<1:
        print('no matches')
    else:
        clsyms = df_lo.option_symbol.values
        theo_expiries = [get_expiry(s) for s in clsyms]
        df_lo['theo'] = theo_expiries
        df_lo['actual'] = df_lo.option_expiry
        df_lo['theo_vs_actual'] = df_lo.actual == df_lo.theo
        len_good = len(df_lo[df_lo.theo_vs_actual])
        len_bad = len(df_lo[~df_lo.theo_vs_actual])
        title = f"{op_sym} good:{len_good} bad:{len_bad}"
        display(dashapp.add_df_title(df_lo[~df_lo.theo_vs_actual],title))


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


...

,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


,option_symbol,underlying_symbol,option_expiry,underlying_expiry,theo,actual,theo_vs_actual


### Test get_expiry against past expirations

In [15]:
if DO_DATABASE_COMPARE:
    pga = db_info.get_db_info()
    futtab = 'sec_schema.underlying_table'
    opttab = 'sec_schema.options_table'


In [16]:
if DO_DATABASE_COMPARE:
    df_pga_expiry = None
    for sym in tqdm(['CL','NG','ES','CB']):
        sql = f"""
        with f0 as
        (
          select max(settle_date) as max_settle_date from {opttab}
          where substring(symbol,1,2) = '{sym}'
        ),
        f1 as 
        (
            select symbol,max(settle_date) settle_date from {opttab}
            where substring(symbol,1,2) = '{sym}'
            group by symbol
        )
        select * from f1 where f1.settle_date < (select max_settle_date from f0)
        """
        df_pga_expiry_temp = pga.get_sql(sql)
        if df_pga_expiry is None:
            df_pga_expiry = df_pga_expiry_temp.copy()
        else:
            df_pga_expiry = df_pga_expiry.append(df_pga_expiry_temp,ignore_index=True)


In [17]:
if DO_DATABASE_COMPARE:
    for sym in ['CL','NG','ES']:
        df_pga_expiry2 = df_pga_expiry[df_pga_expiry.symbol.str.contains(sym)].copy()
        df_pga_expiry2['theo'] = [yyyymmdd_from_dt(get_expiry(s.strip())) for s in df_pga_expiry2.symbol.values]
        df_pga_expiry2 = df_pga_expiry2.rename(columns={'settle_date':'actual'})
        df_pga_expiry2['actual_vs_theo'] = df_pga_expiry2.actual == df_pga_expiry2.theo
        num_ok = len(df_pga_expiry2[df_pga_expiry2.actual_vs_theo])
        num_bad = len(df_pga_expiry2[~df_pga_expiry2.actual_vs_theo])
        title = f"{sym} good:{num_ok} bad:{num_bad}"
        display(dashapp.add_df_title(df_pga_expiry2[~df_pga_expiry2.actual_vs_theo],title))
    